**1. dustfeel 사이트에서 455개의 주소를 beautifulsoup을 통해 따오기**

In [3]:
#dustfeel 사이트에서 455개의 주소를 beautifulsoup을 통해 따온다 
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen


addr_list=[] #주소 저장 리스트
for i in range(46):# 455개 정보 가져옴 (한페이지당 10개의 대기정보가 저장되어 있음)

    url='https://dustfeel.com/station?p='+str(i+1)
    page=urlopen(url)
    soup=BeautifulSoup(page,'html.parser')
    addr_text=soup.find_all('p','addr')
    for j in range(len(addr_text)):
        addr_text[j]=addr_text[j].text#주소 문자열만 따옴(10개)

    addr_list.extend(addr_text)#주소가 저장된 리스트 
    

#주소에서 끝에 대기 부분을 전처리 한다. 
for i in range(len(addr_list)):
    if '도시대기' in addr_list[i]:
        addr_list[i]=addr_list[i].replace('도시대기','')
    elif '도로변대기' in addr_list[i]:
        addr_list[i]=addr_list[i].replace('도로변대기','')
    elif '옥상' in addr_list[i]:
        addr_list[i]=addr_list[i].replace('옥상','')
    elif '교외대기' in addr_list[i]:
        addr_list[i]=addr_list[i].replace('교외대기','')
        

**2. selenium에서 google map에 접속에 455개의 주소를 입력하고 경도 위도를 리스트에 저장한다(크롤링하는데 30분 정도 소요됩니다)**

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  #내용지우기 위해 필요한 모듈
from time import sleep


latitude=[]
longitude=[]

driver = webdriver.Chrome('../driver/chromedriver')
driver.implicitly_wait(3)
driver.get("https://www.google.co.kr/maps/")

for i in range(len(addr_list)):
    select=driver.find_element_by_name('q')
    select.send_keys(addr_list[i])
    driver.find_element_by_xpath("""//*[@id="searchbox-searchbutton"]""").click()
    sleep(2)
    a=driver.current_url
    local=a.split('@')[1]
    latitude.append(local.split(',')[0]) #위도
    longitude.append(local.split(',')[1]) #경도
    sleep(3)
    for j in range(1,len(addr_list[i])+1):
        select.send_keys(Keys.BACKSPACE)
        #sleep(0.5)
    sleep(1)
driver.close() 

ModuleNotFoundError: No module named 'selenium'

**3. 첫 주소만 현재 나의 위치로 저장이 되어, 첫주소만 수정한다**

In [23]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.implicitly_wait(3)
driver.get("https://www.google.co.kr/maps/")
select=driver.find_element_by_name('q')
select.send_keys(addr_list[0])
driver.find_element_by_xpath("""//*[@id="searchbox-searchbutton"]""").click()
sleep(5)
a=driver.current_url
local=a.split('@')[1]
latitude[0]=local.split(',')[0] #위도
longitude[0]=local.split(',')[1] #경도
driver.close()


**4.dataframe에 저장하기**

In [24]:
addr_all={'주소':addr_list,'위도':latitude,'경도':longitude}
df=pd.DataFrame(addr_all)
df.head()


,주소,위도,경도
0,경기 양주시 백석읍 꿈나무로 199꿈나무도서관,37.792424,126.9845236
1,대구 달성군 현풍면 부리 247 (상수도사업본부 달성사업소)(현풍중앙로 144-34...,35.6999534,128.4412756
2,경기 이천시 설성면 신필리산 88-5(전파연구소 입구),37.1387735,127.5449728
3,경기 포천시 삼육사로 2186번길 11-15선단보건지소,37.8533363,127.1570327
4,충북 음성군 금왕읍 무극로370 (금왕공설운동장내),36.9627761,127.5755361


In [25]:
df.to_csv('address.csv',sep=',',encoding='cp949')

**5.folium으로 전국 미세먼지 측정소 지도 표현하고 저장하기**

In [27]:
import folium

map_mw=folium.Map(location=[36.2002,127.054],zoom_start=7) #html의 시작점 


for i in range(len(latitude)):
    folium.Marker([latitude[i],longitude[i]],pop='**').add_to(map_mw)


In [28]:
map_osm.save('dust_station.html')